In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

In [2]:
df = pd.read_csv(r'C:\Users\myeon\Downloads\ml-1m\ratings.dat')

In [3]:
df.head()

,movieId,userId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
cnt = Counter(df['userId']) # 각 userId가 몇 번 등장했는지 count한 뒤에
idx_sorted = sorted(list(cnt.keys()), key=lambda x: -cnt[x]) # 빈도가 높은 순서대로 userId를 정렬

In [5]:
idx = np.zeros(len(df)) # df의 전체 row 길이에 해당하는 vector를 만든 뒤에
for i in idx_sorted[:400]:
    idx += (df['userId'] == i) # 빈도 400등까지의 userId에 해당하는 row에 1씩을 더해준 뒤,
idx = np.array(idx, dtype=bool) # Boolean 타입으로 바꾸어 줌. -> Bool Indexing을 위한 밑작업

In [6]:
new_df = df[idx].iloc[:,:3] # 위의 자료를 이용해 인덱싱을 한 뒤 새로운 df를 만드는데, 마지막 열(timestamp)은 날림
del df # 기존 df는 지워줌. 메모리 절약을 위해 ~^*^~

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_df, test_size=0.3, random_state=42) # 3:7 비율로 test-train split
del new_df; del idx # 메모리 절약...!

In [8]:
utility_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0) # utility matrix 형태로 바꾸어 줌
# 즉, 각 row의 정보가 각 user의 정보이고, 각 column의 정보가 각 movie에 대한 정보가 되도록

In [9]:
utility_matrix.head() # 잘 바뀌었군요

movieId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [10]:
def cos_sim(x, y): # cosine 유사도를 return하는 함수를 define
    return np.dot(x, y) ** 2 / (np.dot(x, x) * np.dot(y, y))

In [11]:
def cos_sim2(x, y, more_than = 5): # 이것은 겹치는 게 5개 이상인 vector 간에 겹치는 column에 한해 cosine 유사도를 반환하는 함수
    idx = (x>0) & (y>0) # 겹치는 부분만 True이고 나머지는 False인 인덱스 벡터를 만듦
    if sum(idx) < more_than: return 0 # minimum 기준을 충족시키지 못하면 유사도가 없는 것으로 반환
    x_n = x[idx]
    y_n = y[idx]
    return np.dot(x_n, y_n) ** 2 / (np.dot(x_n, x_n) * np.dot(y_n, y_n))

In [12]:
from scipy.spatial import distance # Euclidean 거리를 구해주는 모듈

def eucli_sim(x, y, more_than = 5): # 이것은 겹치는 게 5개 이상인 vector 간에 겹치는 column 한정 유클리디언 거리를 반환하는 함수
    idx = (x>0) & (y>0)
    if sum(idx) < more_than: return 100 # 얘는 소소익선이므로, 겹치는 부분이 적으면 크게 반환해주어야 함
    x_n = x[idx]
    y_n = y[idx]
    return distance.euclidean(x_n,y_n)/np.sqrt(len(x_n)) # sqrt(n)으로 나누어주어야 보정이 됨

In [13]:
def find_neighbors(utility_matrix, k, sim_fun): # 주어진 유사도를 기준으로 k개의 neighbor를 구해보자!
    sim_dict = {} # 각 user들의 이웃들을 내포한 딕셔너리로 만들 것임!
    cnt = 0 # 밑에서 반복문 몇 번 돌았는지 확인하려고...
    for i in utility_matrix.index: # utility_matrix의 index들 = userId들
        cnt += 1 # 반복 1회당 카운트 한 번씩 올려주고
        ranking = [(0,0)] * k
        for j in utility_matrix.index.drop(i): # i를 제외한 나머지 userId에 대해 반복
            ranking.append((j, sim_fun(utility_matrix.loc[i], utility_matrix.loc[j]))) # (j, simmilarity between i & j) append
            ranking = sorted(ranking, key=lambda x: -x[1])[:k] # 각 튜플의 뒤의 원소를 기준으로 내림차순한 뒤(유클리디언은 오름차순 해야됨), k개까지만 cut
        sim_dict[i] = ranking # 위에서 만든 리스트를 sim_dict의 key = i에 대해 append함
        if not cnt%10: print(cnt) # 10단위로 카운트 수를 출력... 얼마나 진행되고 있는지 시각적으로 확인코자 하는 수단
    return sim_dict

In [14]:
neighbors = find_neighbors(utility_matrix, 10, cos_sim) # 참 느림... cos_sim2나 eucli로 하면 더 느려집니다 ^*^;;

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400


In [15]:
neighbors # 딕셔너리가 각 user들의 neighbor들을 잘 담고 있음!

{1: [(3114, 0.2071458730879492),
  (1265, 0.20377107980424886),
  (2355, 0.1877773123929886),
  (588, 0.18699269982928762),
  (1580, 0.1697394372616543),
  (1270, 0.16178146754783956),
  (34, 0.15644544141255654),
  (364, 0.15534454133214826),
  (1196, 0.15394467035914897),
  (2571, 0.1535690329497606)],
 2: [(3489, 0.17022953121121406),
  (2161, 0.1216543510404602),
  (367, 0.11983371403938946),
  (2628, 0.11181299321436236),
  (2193, 0.10903294645822573),
  (2617, 0.10336949009224111),
  (736, 0.10330938192188523),
  (2054, 0.1029499965295861),
  (316, 0.10224992316653325),
  (1073, 0.10202478845104919)],
 6: [(2278, 0.15990421956709208),
  (47, 0.1332403359416795),
  (2353, 0.1268688948826786),
  (457, 0.1263084676700625),
  (1213, 0.12569311918265963),
  (50, 0.12547370259118068),
  (1573, 0.12230457147935453),
  (1089, 0.12218885446089685),
  (296, 0.12144522521684534),
  (1729, 0.12132692405985357)],
 10: [(1722, 0.20858271949085566),
  (1370, 0.15971365055996256),
  (3082, 0.157

In [16]:
def prediction(user, movie, neighbors, utility_matrix): # 이제 neighbor들의 평균을 냄으로써 평점을 예측해봅시다
    my_neighbors = neighbors[user] # 지정된 user의 이웃들을 꺼낸 뒤
    rates = []
    for i, _ in my_neighbors:
        r = utility_matrix.loc[i, movie] # 이웃의 해당 영화에 대한 평점을 꺼내옵니다
        if r > 0: rates.append(r) # 정보가 있을 경우에 리스트에 append
    if rates == []: return None # 만약 리스트가 비어있다면, None을 반환 (왜냐면, 이웃 중에 그 영화를 본 사람이 없는 것이므로)
    else: return sum(rates)/len(rates) # 아니라면 평균을 return합시다

In [17]:
def testing(test, neighbors, utility_matrix): # test data에 대해 test하는 함수
    predicted = []
    movies = utility_matrix.columns # train data에 있는 영화 목록
    for i in range(len(test)): # test set에 대해 반복을 돌립니다
        movie = test.iloc[i,0]; user = test.iloc[i,1] # 각 row의 movie, user Id를 따옴
        if (user not in neighbors) or (movie not in movies): # 만약 해당 user의 neighbor data가 없거나, movie가 train movie 목록에 없다면
            predicted.append(None) # 그 데이터에 관해서는 test를 할 수 없음
        else: predicted.append(prediction(user, movie, neighbors, utility_matrix)) # 아니라면 예측을 합니다
    return predicted

In [18]:
test['prediction'] = testing(test, neighbors, utility_matrix) # 위 함수를 활용한 예측 칼럼을 만듦

In [19]:
test['diff'] = test['rating'] - test['prediction'] # 실제와 예측 간의 차이를 담은 column을 만듦

In [20]:
test['diff'].mean() # 예측의 차이가 평균적으로 -0.15만큼 차이가 남!

-0.14593261758308707

In [21]:
test['diff'].std() # rmse는 대략 0.96

0.9585318178970975

In [22]:
neighbors = find_neighbors(utility_matrix, 10, cos_sim2) # cos_sim2로 테스팅 - %% 속도 느림 주의 %%
test['prediction2'] = testing(test, neighbors, utility_matrix)
test['diff2'] = test['rating'] - test['prediction2']
print(test['diff2'].mean(), test['diff2'].std()) # 설명력이 악화됨

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
-0.3371409602384455 0.9826711220725804


In [24]:
# 마지막으로 euclidean 거리로 테스팅 - %% 속도 매우 느림 주의 %%
def find_neighbors2(utility_matrix, k, sim_fun):
    sim_dict = {}
    cnt = 0
    for i in utility_matrix.index:
        cnt += 1
        ranking = [(0,100)] * k # 유클리디언 거리는 소소익선이므로, default 거리를 100으로해서 밀려날 수 있도록 해줌
        for j in utility_matrix.index.drop(i):
            ranking.append((j, sim_fun(utility_matrix.loc[i], utility_matrix.loc[j])))
            ranking = sorted(ranking, key=lambda x: x[1])[:k] # 유클리디언 거리는 오름차순으로 정렬해줘야 가까운 순서임
        sim_dict[i] = ranking
        if not cnt%10: print(cnt)
    return sim_dict

neighbors = find_neighbors2(utility_matrix, 10, eucli_sim)
test['prediction3'] = testing(test, neighbors, utility_matrix)
test['diff3'] = test['rating'] - test['prediction3']
print(test['diff3'].mean(), test['diff3'].std()) # 평균 기준으론 개선되었으나 rmse 기준으로는 동일

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
-0.07841068058817902 0.953334653017283
